# Democracy Taken Hostage — Panel analysis

Uporządkowany notebook do analizy danych V-Dem. Kod został podzielony na etapy (konfiguracja, przygotowanie danych, modelowanie i diagnostyka), aby ułatwić replikację wyników w pracy licencjackiej.

## Środowisko i zależności
Ten blok instaluje brakujące zależności jedynie w środowisku Google Colab oraz ładuje bazowe biblioteki wykorzystywane w dalszych krokach.

In [ ]:
import importlib
from pathlib import Path
from textwrap import dedent

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import jarque_bera, pearsonr, f, chi2
from statsmodels.tools import add_constant
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.outliers_influence import variance_inflation_factor

try:
    from linearmodels import PanelOLS, RandomEffects
except ImportError:
    import sys, subprocess
    if importlib.util.find_spec("google.colab"):
        subprocess.check_call([sys.executable, "-m", "pip", "install", "linearmodels", "stargazer"])
        from linearmodels import PanelOLS, RandomEffects
    else:
        raise

try:
    importlib.import_module("google.colab")
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

plt.style.use("seaborn-v0_8")


## Konfiguracja ścieżek
Zmienna `DATA_PATH` wskazuje lokalizację pliku `V-Dem-CY-Full+Others-v14.csv`. W Colabie domyślnie używany jest katalog na Dysku Google.

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = Path('/content/drive/MyDrive/v-dem/V-Dem-CY-Full+Others-v14.csv')
else:
    DATA_PATH = Path('data/V-Dem-CY-Full+Others-v14.csv')

DATA_PATH


## Funkcje pomocnicze
Zestaw funkcji do ładowania danych, skalowania zmiennych, tworzenia interakcji oraz diagnostyki modeli (VIF, autokorelacja, test Hausmana).

In [ ]:
def min_max_scale(series: pd.Series, invert: bool = False) -> pd.Series:
    scaled = (series - series.min()) / (series.max() - series.min())
    return 1 - scaled if invert else scaled


def load_base_frame(path: Path) -> pd.DataFrame:
    cols = [
        'country_name', 'year', 'v2x_libdem', 'v2cacamps', 'v2x_feduni', 'v2x_regime',
        'v2elparlel', 'v2elmulpar', 'e_gdppc', 'v2xeg_eqdr', 'v2juhcind',
        'v2xeg_eqprotec', 'v2peasbsoc', 'v2smpolsoc'
    ]
    df = pd.read_csv(path, usecols=cols)
    df = df[(df['year'] > 1994) & (df['v2elparlel'] != 3)]
    return df


def add_interactions(df: pd.DataFrame, base: str, partners: dict) -> pd.DataFrame:
    for suffix, col in partners.items():
        df[f"{base}_X_{suffix}"] = df[base] * df[col]
    return df


def compute_vif(frame: pd.DataFrame, features: list) -> pd.DataFrame:
    X = add_constant(frame[features])
    vif_data = pd.DataFrame({
        "Variable": X.columns,
        "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    })
    return vif_data


def wooldridge_autocorr(residuals: pd.Series) -> tuple[float, float]:
    df_resid = residuals.to_frame(name='residual')
    df_resid['unit'] = residuals.index.get_level_values(0)
    df_resid['time'] = residuals.index.get_level_values(1)
    df_resid['lagged_resid'] = df_resid.groupby('unit')['residual'].shift(1)
    df_resid = df_resid.dropna()

    y_aux = df_resid['residual']
    X_aux = add_constant(df_resid['lagged_resid'])
    model_aux = sm.OLS(y_aux, X_aux).fit()
    r_squared = model_aux.rsquared
    n_units = residuals.index.get_level_values(0).nunique()
    f_stat = r_squared / (1 - r_squared) * (n_units - 1)
    p_value = 1 - f.cdf(f_stat, 1, n_units - 1)
    return f_stat, p_value


def hausman_test(fe_res, re_res) -> tuple[float, float]:
    beta_diff = fe_res.params - re_res.params
    cov_diff = fe_res.cov - re_res.cov
    stat = float(beta_diff.T @ np.linalg.inv(cov_diff) @ beta_diff)
    dof = len(beta_diff)
    p_val = 1 - chi2.cdf(stat, dof)
    return stat, p_val


## Przygotowanie danych
- logarytmizacja PKB per capita,
- skalowanie wskaźników partyjnej polaryzacji i zmiennych społecznych,
- tworzenie zmiennych zero-jedynkowych dla systemów wyborczych,
- konstruowanie interakcji zgodnie z dotychczasową analizą.

In [ ]:
base_df = load_base_frame(DATA_PATH)

panel_df = base_df.copy()
panel_df['log_gdp_pc'] = np.log(panel_df['e_gdppc'])
panel_df['polar_n'] = min_max_scale(panel_df['v2cacamps'])
panel_df['pol_of_society_n'] = min_max_scale(panel_df['v2smpolsoc'], invert=True)
panel_df['mulpar_n'] = min_max_scale(panel_df['v2elmulpar'])
panel_df['libdem_logit'] = np.log(panel_df['v2x_libdem'] / (1 - panel_df['v2x_libdem']))

panel_df = panel_df.set_index(['country_name', 'year'])
parlel = pd.get_dummies(panel_df['v2elparlel'], prefix='parlel', drop_first=True)
panel_df = pd.concat([panel_df, parlel], axis=1)

panel_df = panel_df.rename(columns={
    'v2x_libdem': 'libdem',
    'v2x_feduni': 'feduni',
    'v2elparlel': 'parlel',
    'v2x_regime': 'regime',
    'parlel_1.0': 'Prop',
    'parlel_2.0': 'Mixed',
    'v2xeg_eqdr': 'equal',
    'v2juhcind': 'hc_indep',
    'v2xeg_eqprotec': 'eq_rights_prot',
    'v2peasbsoc': 'buss_oport'
})

panel_df['Prop'] = panel_df['Prop'].astype(int)
panel_df['Mixed'] = panel_df['Mixed'].astype(int)

interaction_map = {
    'Equal': 'equal',
    'Prop': 'Prop',
    'Mixed': 'Mixed',
    'gdp_pc_log': 'log_gdp_pc',
    'mulpar': 'mulpar_n',
    'feduni': 'feduni',
    'hc_indep': 'hc_indep',
    'eq_rights_prot': 'eq_rights_prot',
    'buss_oport': 'buss_oport'
}
panel_df = add_interactions(panel_df, 'polar_n', interaction_map)
panel_df['polar_X_year'] = panel_df['polar_n'] * panel_df.index.get_level_values('year')

panel_df = panel_df.dropna()
panel_df


## Statystyki opisowe
Tabela opisowa wybranych zmiennych, eksportowana także do CSV w katalogu roboczym.

In [ ]:
descriptive_columns = ['polar_n', 'Prop', 'Mixed', 'feduni', 'mulpar_n', 'log_gdp_pc', 'equal', 'pol_of_society_n']

stats_table = panel_df[descriptive_columns].describe(percentiles=[0.25, 0.5, 0.75]).T
stats_table.rename(columns={
    'count': 'N', 'mean': 'Mean', 'std': 'St. Dev.', 'min': 'Min',
    '25%': 'Pctl(25)', '50%': 'Median', '75%': 'Pctl(75)', 'max': 'Max'
}, inplace=True)
stats_table['N'] = panel_df[descriptive_columns].count()
stats_table.to_csv('descriptive_statistics.csv')
stats_table


## Korelacje punktowe
Sprawdzenie zależności między polaryzacją a postawami społeczeństwa.

In [ ]:
corr, p_value = pearsonr(panel_df['polar_n'], panel_df['pol_of_society_n'])
print(f"Korelacja Pearsona: {corr:.3f}, p-wartość: {p_value:.4f}")


## Modele panelowe (Newey–West)
Funkcja ułatwia replikację zestawu modeli z pracy: różne konfiguracje zmiennych objaśniających, efekty stałe i czasowe oraz korekta kowariancji.

In [ ]:
def fit_panel_model(df: pd.DataFrame, features: list, label: str):
    X = add_constant(df[features])
    y = df['libdem']
    model = PanelOLS(y, X, entity_effects=True, time_effects=True)
    result = model.fit(cov_type='kernel', kernel='newey-west')
    print(f"
{label}")
    print(result.summary)
    return result

model_specs = {
    'm1': ['polar_n', 'polar_X_feduni', 'polar_X_hc_indep', 'polar_X_Equal', 'polar_X_Prop', 'polar_X_Mixed',
           'polar_X_mulpar', 'polar_X_gdp_pc_log', 'mulpar_n', 'feduni', 'log_gdp_pc', 'equal', 'hc_indep', 'Prop', 'Mixed'],
    'm2': ['pol_of_society_n', 'polar_X_feduni', 'polar_X_Equal', 'polar_X_hc_indep', 'polar_X_Prop', 'polar_X_Mixed',
           'polar_X_mulpar', 'polar_X_gdp_pc_log', 'mulpar_n', 'feduni', 'log_gdp_pc', 'equal', 'hc_indep', 'Prop', 'Mixed'],
    'm3': ['polar_n', 'pol_of_society_n', 'polar_X_feduni', 'polar_X_Equal', 'polar_X_hc_indep', 'polar_X_Prop', 'polar_X_Mixed',
           'polar_X_mulpar', 'polar_X_gdp_pc_log', 'mulpar_n', 'feduni', 'log_gdp_pc', 'equal', 'hc_indep', 'Prop', 'Mixed'],
    'm4': ['polar_n', 'pol_of_society_n', 'mulpar_n', 'feduni', 'hc_indep', 'Prop', 'Mixed'],
    'm5': ['polar_n', 'pol_of_society_n', 'feduni', 'mulpar_n', 'hc_indep', 'log_gdp_pc', 'equal']
}

panel_results = {name: fit_panel_model(panel_df, feats, f"Model {name}") for name, feats in model_specs.items()}


## Tabela wyników (Stargazer)
Zbiorcza tabela modeli z kolejnością zmiennych i skorygowanym R².

In [ ]:
try:
    from stargazer.stargazer import Stargazer
    from IPython.core.display import HTML

    display_order = [
        'polar_n', 'pol_of_society_n', 'polar_X_mulpar', 'polar_X_Mixed', 'polar_X_Prop', 'polar_X_feduni',
        'polar_X_hc_indep', 'polar_X_gdp_pc_log', 'polar_X_Equal', 'mulpar_n', 'Mixed', 'Prop', 'feduni',
        'hc_indep', 'log_gdp_pc', 'equal', 'const'
    ]

    models = [panel_results['m1'], panel_results['m2'], panel_results['m3'], panel_results['m4']]
    adjusted_r2 = []
    for model in models:
        try:
            r2 = model.rsquared_within
            n = model.nobs
            k = model.params.shape[0]
            adjusted_r2.append(round(1 - (1 - r2) * (n - 1) / (n - k - 1), 3))
        except Exception:
            adjusted_r2.append('n/a')

    stargazer = Stargazer(models)
    stargazer.covariate_order(display_order)
    html_output = stargazer.render_html()
    adj_row = "<tr><td><b>Adjusted R² (within)</b></td>" + "".join([f"<td>{val}</td>" for val in adjusted_r2]) + "</tr>"
    insert_pos = html_output.find("</table>")
    html_output = html_output[:insert_pos] + adj_row + html_output[insert_pos:]
    HTML(html_output)
except ImportError:
    print("Stargazer niedostępny w bieżącym środowisku.")


## Przekrój roczny (2000)
Replikacja modeli przekrojowych na danych z jednego roku z korektą heteroskedastyczności.

In [ ]:
df2000 = panel_df.reset_index().set_index('country_name')
df2000 = df2000[df2000['year'] == 2000]

X_full = add_constant(df2000[['polar_n', 'polar_X_Prop', 'polar_X_Mixed', 'polar_X_gdp_pc_log', 'polar_X_mulpar',
                              'polar_X_feduni', 'polar_X_Equal', 'equal', 'feduni', 'mulpar_n', 'Prop', 'Mixed', 'log_gdp_pc']])
y_full = df2000['libdem']
ols_2000_full = sm.OLS(y_full, X_full).fit(cov_type='HC0')

X_trim = add_constant(df2000[['polar_n', 'polar_X_mulpar', 'polar_X_Equal', 'equal', 'feduni', 'mulpar_n', 'log_gdp_pc']])
y_trim = df2000['libdem']
ols_2000_trim = sm.OLS(y_trim, X_trim).fit(cov_type='HC0')

print(ols_2000_full.summary())
print(ols_2000_trim.summary())


## Diagnostyka: heteroskedastyczność

In [ ]:
X_diag = add_constant(panel_df[['polar_n', 'polar_X_feduni', 'polar_X_Equal', 'mulpar_n', 'feduni', 'log_gdp_pc', 'equal']])
y_diag = panel_df['libdem']
ols_diag = sm.OLS(y_diag, X_diag).fit()

bp_stat = het_breuschpagan(ols_diag.resid, ols_diag.model.exog)
print("Breusch–Pagan:", bp_stat)

white_stat = het_white(ols_diag.resid, ols_diag.model.exog)
print("White:", white_stat)


## Diagnostyka: autokorelacja (Wooldridge)

In [ ]:
f_stat, p_val = wooldridge_autocorr(panel_results['m3'].resids)
print(f"Statystyka Wooldridge'a: {f_stat:.3f}, p-wartość: {p_val:.4f}")


## Test Hausmana: efekty stałe vs losowe

In [ ]:
X_haus = panel_df[['polar_n', 'polar_X_feduni', 'polar_X_Equal', 'mulpar_n', 'feduni', 'log_gdp_pc', 'equal']]
y_haus = panel_df['libdem']
fe_res = PanelOLS(y_haus, X_haus, entity_effects=True).fit()
re_res = RandomEffects(y_haus, X_haus).fit()
haus_stat, haus_p = hausman_test(fe_res, re_res)
print(f"Statystyka Hausmana: {haus_stat:.3f}, p-wartość: {haus_p:.4f}")


## Istotność efektów stałych
Test F na istnienie efektów jednostkowych w modelu bazowym.

In [ ]:
fe_model = PanelOLS(y_haus, X_haus, entity_effects=True)
fe_results = fe_model.fit()
fe_results.f_statistic


## Współliniowość (VIF)
Zunifikowane obliczenia VIF dla kluczowych zestawów zmiennych wykorzystywanych w modelach.

In [ ]:
vif_specs = {
    'podstawowy': ['polar_n', 'pol_of_society_n', 'Prop', 'Mixed', 'feduni', 'mulpar_n', 'log_gdp_pc', 'equal', 'hc_indep'],
    'społeczne': ['polar_n', 'pol_of_society_n'],
    'interakcje': ['polar_n', 'polar_X_Prop', 'polar_X_Mixed', 'polar_X_gdp_pc_log', 'polar_X_mulpar', 'polar_X_feduni',
                   'polar_X_Equal', 'equal', 'feduni', 'mulpar_n', 'Prop', 'Mixed', 'log_gdp_pc', 'hc_indep', 'polar_X_hc_indep']
}

vif_results = {name: compute_vif(panel_df, feats) for name, feats in vif_specs.items()}
for name, table in vif_results.items():
    print(f"
VIF — {name}")
    print(table)


## Normalność reszt

In [ ]:
X_norm = add_constant(panel_df[['polar_n', 'polar_X_feduni', 'polar_X_Equal', 'mulpar_n', 'feduni', 'log_gdp_pc', 'equal']])
model_norm = PanelOLS(panel_df['libdem'], X_norm, entity_effects=True).fit()
resid_norm = model_norm.resids

jb_stat, jb_p = jarque_bera(resid_norm)
print(f"Statystyka Jarque–Bera: {jb_stat:.3f}, p-wartość: {jb_p:.4f}")


## Wizualizacja reszt

In [ ]:
sns.histplot(resid_norm, kde=True)
plt.title("Rozkład reszt modelu panelowego")
plt.show()

sm.qqplot(resid_norm, line='s')
plt.title("QQ-plot reszt modelu panelowego")
plt.show()


## Wątek brazylijski
Podgląd ścieżki indeksu liberalnej demokracji w Brazylii.

In [ ]:
brazil = panel_df[panel_df.index.get_level_values('country_name') == 'Brazil'].reset_index()
sns.lineplot(x='year', y='libdem', data=brazil)
plt.title('LDI w Brazylii')
plt.xlabel('Rok')
plt.ylabel('Liberal Democracy Index')
plt.show()

brazil[['year', 'libdem']]
